<a href="https://colab.research.google.com/github/williamedwardhahn/ComplexSystems/blob/main/Complex_Systems_Lab_Random_Access_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Random Access Machine

In [ ]:
import numpy as np

In [ ]:
# --------------- Instruction Set --------------
# hlt 100000       Halt
# ld  11rmmm       Load register r with contents of address mmm.
# sto 12rmmm       Store the contents of register r at address mmm.
# ld# 13rnnn       Load register r with the number nnn.
# ldi 14r00s       Load register r with the memory word addressed by register s.
# add 15r00s       Add contents of register s to register r
# sub 16r00s       Sub contents of register s from register r
# mul 17r00s       Mul contents of register r by register s
# div 18r00s       Div contents of register r by register s
# jmp 200mmm       Jump to location mmm
# jz  21rmmm       Jump to location mmm if register r is zero

In [ ]:
codes = {"hlt":10, "ld":11, "sto":12, "ld#":13, "ldi":14, "add":15, "sub":16, "mul":17, "div":18, "jmp":20, "jz":21}

In [ ]:
def compute(M):
    C = 100
    run = 1
    while run:
        
        I = M[C]
        o = (I // 10000)       # Operation Code
        r = (I // 1000) % 10   # Register
        a = (I %  1000)        # Address

        if   o == 10    : run = False             # stop instruction
        elif o == 11    : R[r] = M[a]             # load Register
        elif o == 12    : M[a] = R[r]             # store Register
        elif o == 13    : R[r] = a                # load Register immediate
        elif o == 14    : R[r] = M[R[a]]          # load Register indexed
        elif o == 15    : R[r] = R[r] + R[a]      # add Register
        elif o == 16    : R[r] = R[r] - R[a]      # sub Register
        elif o == 17    : R[r] = R[r] * R[a]      # mul Register
        elif o == 18    : R[r] = R[r] / R[a]      # div Register
        if   o < 20     : C += 1                  # proceed to next inst
        elif o == 20    : C = a                   # jump unconditionally
        elif o == 21    :                         #   
            if R[r]== 0 : C = a                   # jump if Register zero
            else        : C += 1                  # if not proceed

In [ ]:
def load_program(file):
    with open(file) as f:
        program = f.readlines()
    for i in range(len(program)):
        M[int(program[i][0:3])] = int(program[i][3:10])

# Add Two Numbers

In [ ]:
M = np.zeros(1000, dtype=int)    # Memory
R = np.zeros(10,   dtype=int)    # Registers
C = np.zeros(1,    dtype=int)    # Program Counter
I = np.zeros(1,    dtype=int)    # Instruction Register

In [ ]:
M[100] = 131005  # Load reg 1 with 5
M[101] = 132006  # Load reg 2 with 6
M[102] = 151002  # Add reg2 to reg1
M[103] = 100000  # Stop

In [ ]:
compute(M)

In [ ]:
R

array([ 0, 11,  6,  0,  0,  0,  0,  0,  0,  0])

# Add List of Numbers

In [ ]:
%%writefile addnums.mml
100 130000   reg 0 holds the sum, set to zero
101 131200   put address (200) of 1st number in reg 1
102 132001   reg 2 holds the number one.
103 143001   next number (via reg 1) to reg 3
104 213108   if zero we're done so jump to halt inst
105 150003   otherwise add reg 3 to the sum in reg 0
106 151002   add one (in reg 2) to reg 1 so it points to next number
107 200103   jump back to 103 to get the next number
108 100000   all done so halt. the sum is in reg 0
200 000123   the numbers to add
201 000234
202 000345
203 000000   the end of the list

Overwriting addnums.mml


In [ ]:
M = np.zeros(1000, dtype=int)    # Memory
R = np.zeros(10,   dtype=int)    # Registers
C = np.zeros(1,    dtype=int)    # Program Counter
I = np.zeros(1,    dtype=int)    # Instruction Register

In [ ]:
load_program('addnums.mml')

In [ ]:
123+234+345

702

In [ ]:
compute(M)

In [ ]:
R

array([702, 203,   1,   0,   0,   0,   0,   0,   0,   0])

# Factorial

In [ ]:
def fact(n):
    if n == 1: return 1
    return n * fact(n-1)

In [ ]:
fact(18)

6402373705728000

In [ ]:
%%writefile fact.mml
100 130018     ld# r0,8
101 120117     sto r0,term
102 130001     ld# r0,1
103 120118     sto r0,ans
104 110117     ld  r0,term
105 210115     jz  r0,z2
106 111118     ld  r1,ans
107 112117     ld  r2,term
108 171002     mul r1,r2
109 121118     sto r1,ans
110 111117     ld  r1,term
111 132001     ld# r2,1
112 161002     sub r1,r2
113 121117     sto r1,term
114 200104     jmp  z1
115 110118     ld  r0,ans
116 100000     hlt
117 100000     term 0
118 100000     ans 0

Overwriting fact.mml


In [ ]:
M = np.zeros(1000, dtype=int)    # Memory
R = np.zeros(10,   dtype=int)    # Registers
C = np.zeros(1,    dtype=int)    # Program Counter
I = np.zeros(1,    dtype=int)    # Instruction Register

In [ ]:
load_program('fact.mml')

In [ ]:
compute(M)

In [ ]:
R

array([6402373705728000,                0,                1,
                      0,                0,                0,
                      0,                0,                0,
                      0])

# Assembler

In [ ]:
%%writefile code.mma
go    ld#  r0,0      register 0 will hold the sum, init it
      ld#  r1,nums   register 1 points to the numbers to add
      ld#  r2,1      register 2 holds the number one.
loop  ldi  r3,r1     get next number into register 3
      jz   r3,done   if its zero we're finished
      add  r0,r3     otherwise add it to the sum
      add  r1,r2     add one to register one (next number to load)
      jmp  loop      go for the next one
done  hlt  00        all done. sum is in register 0
nums  123            the numbers to add
      234
      345
        0            end of the list

Overwriting code.mma


In [ ]:
file = 'code.mma'
with open(file) as f:
    program = f.readlines()

In [ ]:
codes = {"hlt":10, "ld":11, "sto":12, "ld#":13, "ldi":14, "add":15, "sub":16, "mul":17, "div":18, "jmp":20, "jz":21}
lookup = {"r0":0,"r1":1,"r2":2,"r3":3,"r4":4,"r5":5,"r6":6,"r7":7,"r8":8,"r9":9}

In [ ]:
def value(s) :
    if not s:
        return 0       
    a = lookup.get(s)         
    if a == None:
        return int(s)  
    else:
        return a

In [ ]:
def Assembler(program):
    C = 100
    for lin in program :
        flds = lin.strip().split()
        if not flds:
            continue        
        if lin[0] > ' ':
            lookup[flds[0]] = C
            if len(flds) > 1:        
                C = C + 1
        else:
            C = C + 1

    C = 100
    for line in program:
        line = line.rstrip()
        flds = line.split()[int(line[0] > ' '):]
        op = codes.get(flds[0])
        if op == None: 
            instruction = int(flds[0])                   
        elif op == 10:
            instruction = int(10)
        else:                                   
            parts = flds[1].split(",")                 
            if len(parts) == 1:
                parts = [0,parts[0]]                   
            instruction = op*10000 + value(parts[0])*1000 + value(parts[1])
        print("%03d %06d" % (C, instruction))

        C = C + 1

    print("%03d %06d" % (C, 100000))
    # print("%03d %06d" % (C+1, 100000))
    # print("%03d %06d" % (C+2, 100000))

In [ ]:
Assembler(program)

100 130000
101 131109
102 132001
103 143001
104 213108
105 150003
106 151002
107 200103
108 000010
109 000123
110 000234
111 000345
112 000000
113 100000


In [ ]:
%%writefile addnums.mml
100 130000
101 131109
102 132001
103 143001
104 213108
105 150003
106 151002
107 200103
108 000010
109 000123
110 000234
111 000345
112 000000
113 100000

Overwriting addnums.mml


In [ ]:
M = np.zeros(1000, dtype=int)    # Memory
R = np.zeros(10,   dtype=int)    # Registers
C = np.zeros(1,    dtype=int)    # Program Counter
I = np.zeros(1,    dtype=int)    # Instruction Register

In [ ]:
load_program('addnums.mml')

In [ ]:
123+234+345

702

In [ ]:
compute(M)

In [ ]:
R

array([702, 112,   1,   0,   0,   0,   0,   0,   0,   0])

# Compiler

In [ ]:
import re, sys
from string import ascii_letters, digits

In [ ]:
def search (a,b) :
	"return index of b within a or -1"
	match = re.search(a,b)
	if match : return match.start()
	else     : return -1

def getToken (prog) :
    "extract next word, number or symbol. return it and rest of prog"
    prog = prog.strip()                        # remove leading whitespace
    if prog == "" : return ['','']             # if no more prog then no token
    if prog[0] in ascii_letters :              # a symbol
        p = search('[^a-zA-z0-9]',prog)        # search for non-alphanumeric
        if p < 0 : return [prog,""]            # return the very last token
        else     : return [prog[:p],prog[p:]]  # or the next alphanumeric token
    elif prog[0] in digits :
        p = search('[^0-9]',prog)              # find first non-numeric
        if p < 0 : return [prog,""]            # return the very last token
        else     : return [prog[:p],prog[p:]]  # or the next numeric token
    else : return [prog[0], prog[1:]]          # or the next (non-alpha) token

def getStat (prog, reg) :
    global nextLabel, vars
    [token, rest] = getToken(prog)          # get statement keyword if any
    if not token : return ['','']           # return if we're all done
    if token == "while" :
        [code1,rest] = getExpr(rest,reg)    # get true/false code to reg
        [code2,rest] = getStat (rest, reg+1) # get main body to next reg

        l1=nextLabel; l2=nextLabel+1; nextLabel=nextLabel+2
        code = "z%d\n%s  jz  r%d,z%d\n%s  jmp  z%d\nz%d\n" % \
                            (l1,code1,reg,l2,code2,l1,l2)
        return [code, rest]
    elif token == "{" :                 # a compound statement. inside {}
        code = ""
        while 1 :
            [tok,rest1] = getToken(rest)    # get statments until "}"
            if not tok    : return ['','']
            if tok == '}' : return [code,rest1]
            [code1,rest] = getStat(rest,reg)
            code = code + code1
    else :
        [second,rest1] = getToken(rest)        # assignment ?
        if second == '=' :
            [code,rest] = getExpr (rest1, reg)
            vars[token] = 1               # remember variable name
            return [code+'  sto r%d,%s\n' % (reg,token), rest]
        else : return getExpr (prog, reg)
    
def getExpr (prog, reg) :
    global nextLabel
    [code1,rest] = getTerm (prog, reg)
    if not code1 : return ['','']
    [opcode,rest1] = getToken(rest)
    if opcode in ['+','*','-','/'] :
        # Use next higher register for 2nd expression
        [code2, rest] = getExpr (rest1, reg+1)
        if opcode == '+' :
            code = '  add r%d,r%d\n' % (reg,reg+1)
        if opcode == '-' :
            code = '  sub r%d,r%d\n' % (reg,reg+1)
        if opcode == '*' :
            code = '  mul r%d,r%d\n' % (reg,reg+1)
        if opcode == '/' :
            code = '  div r%d,r%d\n' % (reg,reg+1)
        return [code1+code2+code, rest]
    else : return [code1, rest]

def getTerm (prog, reg) :
    "Extract number, variable, or nested expression"
    [token, rest] = getToken(prog)    # peek at the first token
    if not token : return ['','']
    if token == "(" :                        # a nested expression
        [code,rest] = getExpr(rest, reg)     # go get it and just make
        if not code : return ['','']
        [token,rest] = getToken(rest)        # make sure closes with ")"
        if token != ")" : return ['','']
        else            : return [code,rest]
    elif token < 'A' :     # got a number - just load to register
        return ['  ld# r%d,%s\n' % (reg,token), rest]
    else :
        return ['  ld  r%d,%s\n' % (reg, token), rest]   # load a variable

In [ ]:
def Compiler(program):
    global vars
    while program :
        [code,program] = getStat(program, 0)
        print(code.rstrip())       
    # print("  hlt")               
    for var in vars.keys():
        print("%s 0" % var)

In [ ]:
nextLabel = 1
vars = {}
Compiler("5*3*8")

  ld# r0,5
  ld# r1,3
  ld# r2,8
  mul r1,r2
  mul r0,r1


In [ ]:
%%writefile code.ma
  ld# r0,5
  ld# r1,3
  ld# r2,8
  mul r1,r2
  mul r0,r1

Overwriting code.ma


In [ ]:
file = 'code.ma'
with open(file) as f:
    program = f.readlines()
program

['  ld# r0,5\n',
 '  ld# r1,3\n',
 '  ld# r2,8\n',
 '  mul r1,r2\n',
 '  mul r0,r1']

In [ ]:
Assembler(program)

100 130005
101 131003
102 132008
103 171002
104 170001
105 100000


In [ ]:
%%writefile code.ml
100 130005
101 131003
102 132008
103 171002
104 170001
105 100000

Overwriting code.ml


In [ ]:
M = np.zeros(1000, dtype=int)    # Memory
R = np.zeros(10,   dtype=int)    # Registers
C = np.zeros(100,  dtype=int)    # Program Counter
I = np.zeros(1,    dtype=int)    # Instruction Register

In [ ]:
load_program('code.ml')

In [ ]:
compute(M)

In [ ]:
R

array([120,  24,   8,   0,   0,   0,   0,   0,   0,   0])

In [ ]:
5*3*8

120

In [ ]:
nextLabel = 1
vars = {"a":0,"b":0}
Compiler("c=(b+3)*a")

  ld  r0,b
  ld# r1,3
  add r0,r1
  ld  r1,a
  mul r0,r1
  sto r0,c
a 0
b 0
c 0


In [ ]:
%%writefile code.ma
  ld  r0,b
  ld# r1,3
  add r0,r1
  ld  r1,a
  mul r0,r1
  sto r0,c
a 0
b 0
c 0

Overwriting code.ma


In [ ]:
file = 'code.ma'
with open(file) as f:
    program = f.readlines()
program

['  ld  r0,b\n',
 '  ld# r1,3\n',
 '  add r0,r1\n',
 '  ld  r1,a\n',
 '  mul r0,r1\n',
 '  sto r0,c\n',
 'a 0\n',
 'b 0\n',
 'c 0']

In [ ]:
Assembler(program)

100 110107
101 131003
102 150001
103 111106
104 170001
105 120108
106 000000
107 000000
108 000000
109 100000


In [ ]:
%%writefile code.ml
100 110107
101 131003
102 150001
103 111106
104 170001
105 120108
106 000015   
107 000022   
108 000000   
109 100000

Overwriting code.ml


In [ ]:
M = np.zeros(1000, dtype=int)    # Memory
R = np.zeros(10,   dtype=int)    # Registers
C = np.zeros(100,  dtype=int)    # Program Counter
I = np.zeros(1,    dtype=int)    # Instruction Register

In [ ]:
load_program('code.ml')

In [ ]:
compute(M)

In [ ]:
M[108]

375

In [ ]:
a = 15
b = 22
c = (b+3)*a
c

375

In [ ]:
8*7*6*5*4*3*2*1

40320

In [ ]:
%%writefile fact.mh
term = 8
ans  = 1
while term { ans=ans*term  term=term-1 }
ans

Overwriting fact.mh


In [ ]:
file = 'fact.mh'
with open(file) as f:
    program = f.readlines()
program = ' '.join(program)

In [ ]:
nextLabel = 1
vars = {}

In [ ]:
Compiler(program)

  ld# r0,8
  sto r0,term
  ld# r0,1
  sto r0,ans
z1
  ld  r0,term
  jz  r0,z2
  ld  r1,ans
  ld  r2,term
  mul r1,r2
  sto r1,ans
  ld  r1,term
  ld# r2,1
  sub r1,r2
  sto r1,term
  jmp  z1
z2
  ld  r0,ans
term 0
ans 0


In [ ]:
# 100 030005     ld# r0,5
# 101 020117     sto r0,term
# 102 030001     ld# r0,1
# 103 020118     sto r0,ans
#              z1
# 104 010117     ld  r0,term
# 105 110115     jz  r0,z2
# 106 011118     ld  r1,ans
# 107 012117     ld  r2,term
# 108 071002     mul r1,r2
# 109 021118     sto r1,ans
# 110 011117     ld  r1,term
# 111 032001     ld# r2,1
# 112 061002     sub r1,r2
# 113 021117     sto r1,term
# 114 100104     jmp  z1
#              z2
# 115 010118     ld  r0,ans
# 116 000000     hlt
# 117 000000   term 0
# 118 000000   ans 0

In [ ]:
vars

{'ans': 1, 'term': 1}

In [ ]:
%%writefile code.ma
     ld# r0,8
     sto r0,term
     ld# r0,1
     sto r0,ans
 z1  ld r0,term
     jz  r0,z2
     ld  r1,ans
     ld  r2,term
     mul r1,r2
     sto r1,ans
     ld  r1,term
     ld# r2,1
     sub r1,r2
     sto r1,term
     jmp  z1
 z2  ld  r0,ans
z1 0
z2 0
term 0
ans 0

Overwriting code.ma


In [ ]:
file = 'code.ma'
with open(file) as f:
    program = f.readlines()

In [ ]:
program

['     ld# r0,8\n',
 '     sto r0,term\n',
 '     ld# r0,1\n',
 '     sto r0,ans\n',
 ' z1  ld r0,term\n',
 '     jz  r0,z2\n',
 '     ld  r1,ans\n',
 '     ld  r2,term\n',
 '     mul r1,r2\n',
 '     sto r1,ans\n',
 '     ld  r1,term\n',
 '     ld# r2,1\n',
 '     sub r1,r2\n',
 '     sto r1,term\n',
 '     jmp  z1\n',
 ' z2  ld  r0,ans\n',
 'z1 0\n',
 'z2 0\n',
 'term 0\n',
 'ans 0']

In [ ]:
Assembler(program)

In [ ]:
%%writefile code.ml
100 130008
101 120116
102 130001
103 120117
104 110116
105 210115
106 111117
107 112116
108 171002
109 121117
110 111116
111 132001
112 161002
113 121116
114 200104
115 110117
116 100000
117 100000
118 100000

Overwriting code.ml


In [ ]:
M = np.zeros(1000, dtype=int)    # Memory
R = np.zeros(10,   dtype=int)    # Registers
C = np.zeros(100,  dtype=int)    # Program Counter
I = np.zeros(1,    dtype=int)    # Instruction Register

In [ ]:
load_program('code.ml')

In [ ]:
compute(M)

In [ ]:
R

array([40320,     0,     1,     0,     0,     0,     0,     0,     0,
           0])